## Long Form Summarization using `ChatGPT` (16k) and `DocAI`

#### Imports 

In [ ]:
from langchain.prompts.chat import SystemMessagePromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from google.api_core.client_options import ClientOptions
from langchain.prompts.chat import ChatPromptTemplate
from concurrent.futures import ThreadPoolExecutor
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from google.cloud import documentai
from pypdf import PdfWriter
from pypdf import PdfReader 
from tqdm import tqdm
import tiktoken
import vertexai
import requests
import logging
import time
import json
import yaml
import os

##### Setup logging 

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

#### Essentials 

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './../credentials/vai-key.json'
access_token = !gcloud auth print-access-token

In [ ]:
with open('./../credentials/oai-key.yml', 'rb') as f:
    credentials = yaml.safe_load(f)
    
api_key = credentials['key']
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
PROJECT_ID = 'arun-genai-bb'
LOCATION = 'us-central1'
MODEL_NAME = 'gpt-3.5-turbo-16k'
ENCODING_NAME = 'cl100k_base'
CONTEXT_LENGTH = 16385  # GPT-4
STREAMING_API_URL = f'https://us-central1-aiplatform.googleapis.com/ui/projects/{PROJECT_ID}/locations/us-central1/publishers/google/models/{MODEL_NAME}:serverStreamingPredict'
DOCAI_PROCESSOR_NAME = 'projects/390991481152/locations/us/processors/ad9557a5be49204e'  # copy from notebook 00
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
client_options = ClientOptions(api_endpoint=f'us-documentai.googleapis.com')
docai_client = documentai.DocumentProcessorServiceClient(client_options=client_options)

In [ ]:
encoder = tiktoken.get_encoding(ENCODING_NAME)
logger.info(f'Using encoder=={encoder.name}')

In [ ]:
model = ChatOpenAI(openai_api_key=api_key, 
                   model_name=MODEL_NAME, 
                   temperature=0.0, 
                   max_tokens=512)
logger.info(f'Using model=={model.model_name}')

#### Use Google DocumentAI to process input PDF

##### Break PDF into smaller PDFs for OCR

In [ ]:
LOCAL_INPUT_DIR = './DATA/INPUT'
LOCAL_OUTPUT_DIR = './DATA/OUTPUT'
FILE_NAME = 'file-2'

In [ ]:
reader = PdfReader(f'{LOCAL_INPUT_DIR}/{FILE_NAME}.pdf') 
pages = {}

for i, page in enumerate(reader.pages):
    pages[i] = page

In [ ]:
n = len(reader.pages)
d = 15  # docai has a current constraint of 15 pages per document 
for i in range(0, n, d):
    writer = PdfWriter()
    for j in range(i, i+d):
        if j < n:
            writer.add_page(pages[j])
    os.makedirs(f'{LOCAL_INPUT_DIR}/{FILE_NAME}/PARTS/', exist_ok=True)
    with open(f'{LOCAL_INPUT_DIR}/{FILE_NAME}/PARTS/{FILE_NAME}_{i+1}-{i+d}.pdf', 'wb') as f:
        writer.write(f)

In [ ]:
def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document's text. This function converts
    offsets to a string.
    """
    # If a text segment spans several lines, it will be stored in different text segments.
    return ''.join(text[int(segment.start_index): int(segment.end_index)] for segment in layout.text_anchor.text_segments)

In [ ]:
def get_file_paths(dir_name: str) -> list:
    file_paths = []
    for file_name in os.listdir(dir_name):
        if os.path.isfile(os.path.join(dir_name, file_name)):
            file_path = os.path.join(dir_name, file_name)
            file_paths.append(file_path)
    return file_paths

In [ ]:
def ocr_docai(file_path: str) -> dict:
    pages_map = {}

    with open(file_path, 'rb') as f:
        pdf = f.read()
        raw_document = documentai.RawDocument(content=pdf, mime_type='application/pdf')
        request = documentai.ProcessRequest(name=DOCAI_PROCESSOR_NAME, raw_document=raw_document)
        response = docai_client.process_document(request=request)
        text = response.document.text
        file_name = file_path.split('/')[-1]
        page_number = int(file_name.split('.')[0].split('-')[-1])
        for page in response.document.pages:
            page_text = []
            for paragraph in page.paragraphs:
                paragraph_text = layout_to_text(paragraph.layout, text)
                page_text.append(paragraph_text)
            pages_map[page_number] = ''.join(page_text)
            page_number += 1
    return pages_map

In [ ]:
%%time 

input_dir = f'./DATA/INPUT/{FILE_NAME}/PARTS/'
file_paths = get_file_paths(input_dir)
    
pages_map_list = []
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:  
    pages_map_list = list(tqdm(executor.map(ocr_docai, file_paths)))

merged_dict = {k: v for d in pages_map_list for k, v in d.items()}   
sorted_pages_map = dict(sorted(merged_dict.items()))

pages = []
for _, page_text in sorted_pages_map.items():
    pages.append(page_text)

Save concatenated pages as txt for later use (if needed)

In [ ]:
extracted_pages = ''.join(pages)
os.makedirs(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/OAI/3.5', exist_ok=True)
with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/OAI/3.5/{FILE_NAME}.txt', 'w') as out:
    out.write(extracted_pages)

In [ ]:
def get_total_tokens(contexts: list) -> int:
    total_tokens = 0
    for context in contexts:
        n_tokens = len(encoder.encode(context))
        total_tokens += n_tokens 
    return total_tokens

In [ ]:
total_tokens = get_total_tokens([extracted_pages])
logger.info(f'Total tokens in the input doc = {total_tokens}')

In [ ]:
def get_max_tokens_per_page(contexts: list) -> list:
    max_tokens_per_page = 0
    for context in contexts:
        n_tokens = len(encoder.encode(context))
        if n_tokens > max_tokens_per_page:
            max_tokens_per_page = n_tokens
    return max_tokens_per_page

#### Map Reduce 1

In [ ]:
model = ChatOpenAI(openai_api_key=api_key, 
                   model_name=MODEL_NAME, 
                   temperature=0.0, 
                   max_tokens=256)

In [ ]:
def get_summary(chunk: str) -> str:
    template = "You are a Financial Regulations & Derivatives Expert"
    system_message = SystemMessagePromptTemplate.from_template(template)

    template = "Summarize the following information into five brief sentences in English, capturing the essential details.\n\n{chunk}"
    human_message = HumanMessagePromptTemplate.from_template(template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    prompt = chat_prompt.format_prompt(chunk=chunk).to_messages()

    response = model(prompt)
    return response.content

In [ ]:
%%time 

MAX_OUTPUT_TOKENS = 256
CONTEXTS_PER_CALL = 5  # process 5 pages per API call


def reduce(contexts: list) -> list:
    partitions = []
    max_input_tokens = CONTEXT_LENGTH - MAX_OUTPUT_TOKENS
    logger.info(f'Max input tokens allowed per API call = {max_input_tokens}')
    max_tokens_per_page = get_max_tokens_per_page(contexts)
    logger.info(f'Max tokens per page = {max_tokens_per_page}')
    logger.info(f'Processing {CONTEXTS_PER_CALL} pages per API call')
    
    for i in range(0, len(contexts), CONTEXTS_PER_CALL):
        partitions.append(contexts[i: i+CONTEXTS_PER_CALL])

    chunks = []
    for partition in partitions:
        chunks.append('\n'.join(partition))

    reduced_contexts = []

    # max_workers can result in running over quota limits for invocation | current limit for text bison is 60/min
    # for our experiments, we set max_workers=4 cores without any limit breach
    with ThreadPoolExecutor(max_workers=4) as executor:  
        reduced_contexts = list(tqdm(executor.map(get_summary, chunks),  total=len(chunks)))
    return reduced_contexts


In [ ]:
logger.info(f'Number of pages to process = {len(pages)}')
summaries = reduce(pages)
logger.info(f'Number of generated summaries = {len(summaries)}')
n_tokens = get_total_tokens(summaries)
logger.info(f'Total number of tokens in generated summaries = {n_tokens}')

In [ ]:
logger.info(summaries[5])
logger.info('-' * 100)
logger.info(summaries[15])
logger.info('-' * 100)
logger.info(summaries[20])

##### Persist internediate summaries (Map Reduce 1) to local disk

In [ ]:
logger.info(f'Total number of summaries = {len(summaries)}')

In [ ]:
for i, summary in enumerate(summaries):
    os.makedirs(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/OAI/3.5/MAP_REDUCE_1/', exist_ok=True)
    with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/OAI/3.5/MAP_REDUCE_1/summary-{i}.txt', 'w') as f:
        f.write(summary)

#### Map Reduce 2

In [ ]:
model = ChatOpenAI(openai_api_key=api_key, 
                   model_name=MODEL_NAME, 
                   temperature=0.0, 
                   max_tokens=2048)

def get_summary(context: str) -> str:
    template = "You are a Financial Regulations & Derivatives Expert"
    system_message = SystemMessagePromptTemplate.from_template(template)

    template = """"For the context below, create a consolidated refined short summary with the most important pointers only.\n\n{context}\n\nDo not repeat pointers. Breakdown the summary into SECTIONS. Make it crisp and concise."""
    human_message = HumanMessagePromptTemplate.from_template(template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    prompt = chat_prompt.format_prompt(context=context).to_messages()

    response = model(prompt)
    return response.content

In [ ]:
%%time 

CONTEXTS_PER_CALL = 25  # process 50 summaries per API call

def reduce(contexts: list) -> list:
    partitions = []
    max_input_tokens = CONTEXT_LENGTH - MAX_OUTPUT_TOKENS
    logger.info(f'Max input tokens allowed per API call = {max_input_tokens}')
    max_tokens_per_page = get_max_tokens_per_page(contexts)
    logger.info(f'Max tokens per page = {max_tokens_per_page}')
    logger.info(f'Processing {CONTEXTS_PER_CALL} pages per API call')
    
    for i in range(0, len(contexts), CONTEXTS_PER_CALL):
        partitions.append(contexts[i: i+CONTEXTS_PER_CALL])

    chunks = []
    for partition in partitions:
        chunks.append('\n'.join(partition))
    logger.info(f'Total number of chunks of summaries = {len(chunks)}')

    reduced_contexts = []
    for chunk in tqdm(chunks):
        summary = get_summary(chunk)
        reduced_contexts.append(summary)
        time.sleep(1)
    return reduced_contexts

In [ ]:
reduced_summaries = reduce(summaries)

In [ ]:
logger.info(reduced_summaries[0])
logger.info('-' * 100)
logger.info(reduced_summaries[1])
logger.info('-' * 100)
logger.info(reduced_summaries[2])
logger.info('-' * 100)

##### Persist internediate summaries (Map Reduce 2) to local disk

In [ ]:
logger.info(f'Total number of summaries after map reduce 2 = {len(reduced_summaries)}')

In [ ]:
for i, summary in enumerate(reduced_summaries):
    os.makedirs(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/OAI/3.5/MAP_REDUCE_2/', exist_ok=True)
    with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/OAI/3.5/MAP_REDUCE_2/summary-{i}.txt', 'w') as f:
        f.write(summary)

In [ ]:
consolidated_summaries = '\n'.join(reduced_summaries)
logger.info(get_total_tokens([consolidated_summaries]))

In [ ]:
logger.info(consolidated_summaries)

#### Final Consolidation

In [ ]:
model = ChatOpenAI(openai_api_key=api_key, 
                   model_name=MODEL_NAME, 
                   temperature=0.0, 
                   max_tokens=1536)

def get_summary(context: str) -> str:
    template = "You are a Financial Regulations & Derivatives Expert"
    system_message = SystemMessagePromptTemplate.from_template(template)

    template = """"Given the context below, combine and merge duplicate sections and pointers.\n\n{context}\nAdd SECTIONS and bullets wherever needed. Clean rewrite and re-number sections."""
    human_message = HumanMessagePromptTemplate.from_template(template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    prompt = chat_prompt.format_prompt(context=context).to_messages()

    response = model(prompt)
    return response.content

In [ ]:
final_summary = get_summary(consolidated_summaries)
logger.info(final_summary)

In [ ]:
MAX_OUTPUT_TOKENS = 4096
max_input_tokens = CONTEXT_LENGTH - MAX_OUTPUT_TOKENS
logger.info(f'Max input tokens allowed per API call = {max_input_tokens}')
logger.info(f'Total tokens in final summary = {get_total_tokens([final_summary])}')

##### Persist final summary to local disk

In [ ]:
with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/OAI/3.5/final-summary.txt', 'w') as f:
    f.write(final_summary)

##### Create a filtered summary with all the proposed changes on the `Processing of Derivative Contracts`

In [ ]:
model = ChatOpenAI(openai_api_key=api_key, 
                   model_name=MODEL_NAME, 
                   temperature=0.0, 
                   max_tokens=MAX_OUTPUT_TOKENS)

def get_summary(context: str) -> str:
    template = "You are a Financial Regulations & Derivatives Expert"
    system_message = SystemMessagePromptTemplate.from_template(template)

    template = """"Given the SUMMARY below, extract and refine all proposed changes related to the processing of derivative contracts into a separate list. Create a concise and detailed summary with clear pointers.\n\n{context}"""
    human_message = HumanMessagePromptTemplate.from_template(template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    prompt = chat_prompt.format_prompt(context=context).to_messages()

    response = model(prompt)
    return response.content

In [ ]:
proposed_changes_summary = get_summary(consolidated_summaries)
logger.info(proposed_changes_summary)

In [ ]:
MAX_OUTPUT_TOKENS = 1536
max_input_tokens = CONTEXT_LENGTH - MAX_OUTPUT_TOKENS
logger.info(f'Max input tokens allowed per API call = {max_input_tokens}')
logger.info(f'Total tokens in final summary = {get_total_tokens([proposed_changes_summary])}')

In [ ]:
with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/OAI/3.5/proposed-changes-summary.txt', 'w') as f:
    f.write(proposed_changes_summary)